# Importing dependencies
**Pip, in case of no Module errors**

*Note: If you are using collab, this might not work, do it in your local machine*

**If you are making dataset from internet**

In [ ]:
!pip install bing-image-downloader

In [ ]:
from bing_image_downloader import downloader

**If you are making dataset from your own camera**

In [ ]:
import cv2
import os
import time
import labelme
# if you want to name your captured images with unique id's you could use uuid

# Data collection

In [ ]:
data_dir = 'data'
num_Img = 5

In [ ]:
# making data folders
if os.path.exists(data_dir):
    if os.path.exists(os.path.join(data_dir,'image')):
        pass
    else:
        os.mkdir(os.path.join(data_dir,'image'))
        
    if os.path.exists(os.path.join(data_dir,'label')):
        pass
    else:
        os.mkdir(os.path.join(data_dir,'label'))
else:
    os.mkdir(os.path.join(data_dir))
    os.mkdir(os.path.join(data_dir,'image'))
    os.mkdir(os.path.join(data_dir,'label'))

In [ ]:
cap = cv2.VideoCapture(0)
for img_Num in range(num_Img):
    ret, frame = cap.read()
    cv2.imwrite(os.path.join(data_dir,'image','cap_img_{}.jpg'.format(img_Num)), frame)
    cv2.imshow('image',frame)
    time.sleep(5)
    if cv2.waitKey(10) and 0xFF == 'q':
        break
cap.release()
cv2.destroyAllWindows()

**Load images fom bing search engine**

In [ ]:
img_dir = os.path.join(data_dir,'image')
downloader.download('People', limit = 50, output_dir = img_dir, adult_filter_off = False)

# Labeling

In [ ]:
!labelme

In [ ]:
# check
len(os.listdir(img_dir)), len(os.listdir(os.path.join(data_dir,'label')))

In [ ]:
pip install albumentations

In [ ]:
import albumentations as A

In [ ]:
sample_aug = A.Compose([A.RandomBrightness(p=0.5),
                        A.RandomContrast(p=0.5),
                        A.RandomGamma(p=0.2),
                        A.RandomSunFlare(p=0.1),
                        A.RandomGravel(p=0.1),
                        A.RandomRain(p=0.3),
                        A.RandomToneCurve(p=0.3),
                        A.HorizontalFlip(p=0.5),
                        A.VerticalFlip(p=0.5),
                        A.RandomRotate90(p=0.3)],
                       keypoint_params=A.KeypointParams(format='xy', label_fields=['class_labels']),)

In [ ]:
import json
import matplotlib.pyplot as plt

In [ ]:
img = cv2.imread(os.path.join(data_dir,'image','cap_img_0.jpg'))
with open(os.path.join(data_dir,'label','cap_img_0.json')) as f:
    annot = json.load(f)
    stuff = annot['shapes']
    aug = sample_aug(image = img,
                     keypoints = [tuple(stuff[0]['points'][0]),tuple(stuff[1]['points'][0])], 
                     class_labels=[stuff[0]['label'], stuff[1]['label']])
    plt.imshow(cv2.cvtColor(aug['image'], cv2.COLOR_BGR2RGB))
    print(aug['keypoints'],aug['class_labels'])

In [ ]:
!git clone https://github.com/heartexlabs/labelImg.git

In [ ]:
!pip install pyqt5 lxml labelimg

In [ ]:
!labelimg

# Reading frames from the video

In [ ]:
import cv2
cap=cv2.VideoCapture("C:\Users\raraj\Downloads\vlc-record-2023-07-03-15h56m40s-_101_SMOKE_NVR_ch76_main_20230702113100_20230702113303.dav-.mp4")


# Removing alternate images in data

In [ ]:
import os

def remove_alternate(data_path):
    init = 0
    for i in os.listdir(data_path):
        if init%2==0:
            os.remove(os.path.join(data_path,i))
        init+=1

In [ ]:
remove_alternate(os.path.join('c:\\','Users','raraj','Downloads','annotations','dataset2'))

# Counting and Pointing out stabel objects

## Reading text annotation files

In [ ]:
import os

In [ ]:
def read_annot(label_path):
    file = open(label_path, "r")
    f = file.readlines()
    info = {}
    coord_li = []
    for i in f:
        li = i.split(' ')
        label = int(li[0])
        coords = list(map(float, li[1:]))
        if label not in info:
            info.update({label:[coords]})
        else:
            info[label].append(coords)
    return info

In [ ]:
def write_annot(label_path):
    file = open(file_path, "a")
    file.write('\nmatter added  successfully')

In [ ]:
read_txt(os.path.join("C:\\","Users","raraj","Downloads","annotations","dataset2","exp.txt"))

# Object detection data augmentation

In [1]:
import os
import albumentations as A
import cv2

In [5]:
data_path = "D:/annotations/failed"
#os.path.join('c:\\','Users','raraj','Downloads','annotations','open_source')
new_path = "D:/annotations/Fire_Zawar_mines/Augmented"
#os.path.join('c:\\','Users','raraj','Downloads','annotations','Flipped')

In [6]:
aug1 = A.Compose([A.augmentations.geometric.transforms.Affine(p=0.3,scale=[0.25,1.75]),
                 A.augmentations.geometric.transforms.Affine(p=0.2,rotate=[-45,45])],
                bbox_params=A.BboxParams(format='yolo'))
aug2 = A.Compose([A.HorizontalFlip(p=1)],
                 bbox_params=A.BboxParams(format='yolo'))

In [7]:
c=0
for i in os.listdir(data_path):
    annot=[]
    # getting the image
    if i.split('.')[1]=='jpg':
        label=os.path.join(data_path,f'{i.split(".")[0]}.txt')
        if os.path.exists(label):
            with open(label,'r') as l:
                labels = l.readlines()
                img = cv2.imread(os.path.join(data_path,i))
            for j in labels:
                li = j.split(" ")
                label = int(li[0])
                coord = list(map(float, li[1:]))
                coord.append(label)
                annot.append(coord)
            try:
                sample_aug = aug1(image=img, bboxes=annot)
                flipped = aug2(image=img, bboxes=annot)
                cv2.imwrite(os.path.join(new_path,f'{i.split(".")[0]}_afine_transform.jpg'),sample_aug['image'])
                cv2.imwrite(os.path.join(new_path,f'{i.split(".")[0]}_flipped.jpg'),flipped['image'])
                with open(os.path.join(new_path,f'{i.split(".")[0]}_afine_transform.txt'),'w') as new:
                    for k in range(len(sample_aug['bboxes'])):
                        new.write(f"{sample_aug['bboxes'][k][-1]} {sample_aug['bboxes'][k][0]} {sample_aug['bboxes'][k][1]} {sample_aug['bboxes'][k][2]} {sample_aug['bboxes'][k][3]} \n")
                new.close()
                with open(os.path.join(new_path,f'{i.split(".")[0]}_flipped.txt'),'w') as flip:
                    for k in range(len(flipped['bboxes'])):
                        flip.write(f"{flipped['bboxes'][k][-1]} {flipped['bboxes'][k][0]} {flipped['bboxes'][k][1]} {flipped['bboxes'][k][2]} {flipped['bboxes'][k][3]} \n")
                flip.close()
            except:
                c += 1
print(c)

0


In [ ]:
sample_aug.keys()

In [2]:
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'matplotlib'

In [ ]:
plt.imshow(sample_aug['image'])

# Splitting data

In [1]:
import os
def split_data(data_path,new_path,n_splits,annoted=False,annot_format=None):
    if annoted:
        assert annot_format!=None
        data_list = os.listdir(data_path)
        for _ in data_list:
            if _.split(".")[1]==annot_format:
                data_list.remove(_)
        split_value = len(data_list)//n_splits
    else:
        data_list = os.listdir(data_path)
        split_value = len(data_list)//n_splits
    splitted_li = []
    start_limit = 0
    end_limit = split_value
    for i in range(1,split_value+1):
        try:
            splitted_li.append(data_list[start_limit:end_limit])
            start_limit = end_limit
            end_limit = split_value*(i+1)
        except IndexError:
            splitted_li.append(data_list[start_limit:])
    for n,j in enumerate(splitted_li):
        cur_path = os.path.join(new_path,f'{n}')
        for k in j:
            if os.path.exists(cur_path):
                os.replace(os.path.join(data_path,k),os.path.join(cur_path,k))
            else:
                os.mkdir(cur_path)
                os.replace(os.path.join(data_path,k),os.path.join(cur_path,k))
            if annoted:
                if os.path.exists(os.path.join(data_path,f'{k.split(".")[0]}.{annot_format}')):
                    os.replace(os.path.join(data_path,f'{k.split(".")[0]}.{annot_format}'),os.path.join(cur_path,f'{k.split(".")[0]}.{annot_format}'))

In [2]:
data_path = 'D:/annotations/fire_comm'
new_path = 'D:/annotations/fire_comm'
split_data(data_path,new_path,3,True,'txt')

In [16]:
sample="B27_2023-05-31_12-17-38.mp426477.jpg"
sample[::-1].split(".",1)[0][::-1]

'jpg'

In [16]:
import json
import os

json_path = "C:/Users/raraj/Downloads/test.json"
json_file = open(json_path)
type(json.load(json_file))

dict

In [14]:
string = """{
    "Name": "Jennifer Smith",
    "Contact Number": 7867567898,
    "Email": "jen123@gmail.com",
    "Hobbies":["Reading", "Sketching", "Horse Riding"]
    }"""

In [17]:
x=json.loads(string)
type(x)

dict

In [111]:
z={(1,2):2,2:4,3:6,4:'?? ??\?'}
n=json.dumps(z, skipkeys=True, ensure_ascii=True, separators =(",", "="))
n

'{"2"=4,"3"=6,"4"="?? ??\\\\?"}'

n = float('nan')
print(type(n))

In [137]:
import logging

fmt = '%(asctime)s: %(message)s'
logging.basicConfig(format=fmt, filename="C:/Users/raraj/test.log", level=logging.INFO, filemode='a')
logging.info('This will get logged')

# Keypoint data sugmentation

In [55]:
'''
Created on 15-May-2023

@author: raraj
'''
import cv2
import json
import albumentations as A
import os


augmentor =    A.Compose([A.RandomBrightness(p=0.5),
                    A.RandomContrast(p=0.5),
                    A.RandomGamma(p=0.2),
                    A.RandomFog(p=0.1),
                    A.RandomRain(p=0.3),
                    A.RandomToneCurve(p=0.3),
                    A.HorizontalFlip(p=0.5),
                    A.VerticalFlip(p=0.5),
                    A.RandomRotate90(p=0.3),
                    A.augmentations.geometric.transforms.Affine(p=0.3,scale=[0.25,0.75]),
                    A.augmentations.geometric.transforms.Affine(p=0.3,scale=[1.25,1.75]),
                    A.augmentations.geometric.transforms.Affine(p=0.3,rotate=[-15,15]) ],
                   keypoint_params=A.KeypointParams(format='xy', label_fields=['point_labels','image_labels' ]),)

label_dir = r"C:\Users\raraj\Downloads\annotations\data\label"
labels = os.listdir(label_dir)
root_dir = r"C:\Users\raraj\Downloads\annotations"
for label in labels:
    with open(os.path.join(label_dir,label)) as f:
        annot = json.load(f)
        stuff = annot['shapes']
        img_path = os.path.join('/',root_dir,'data', 'image',f'{label.split(".")[0]}.jpg')
        img = cv2.imread(img_path)
        print(img_path, img)
        if len(stuff)>2:
            coords = [tuple(stuff[0]['points'][0]),tuple(stuff[1]['points'][0])]
            classes = [stuff[0]['label'], stuff[1]['label']]
            img_type = [stuff[2]['label']]
        else:
            coords = [tuple(stuff[0]['points'][0])]
            classes = [stuff[0]['label']]
            img_type = [stuff[1]['label']]
        for i in range(60):
            annotation = {}
            new_aug = augmentor(image=img,keypoints=coords,point_labels=classes, image_labels = img_type)
            annotation['image']=f'{img_path.split(".")[0]}_{i}.jpg'
            annotation['points']=new_aug['keypoints']
            annotation['point_labels']=new_aug['point_labels']
            annotation['image_label']=new_aug['image_labels']
            cv2.imwrite(f'{img_path.split(".")[0]}_{i}.jpg',new_aug['image'])
            with open(os.path.join('/',root_dir,'data','label',f'{label.split(".")[0]}_{i}.json'), 'w') as o:
                json.dump(annotation,o)

C:\Users\raraj\Downloads\annotations\data\image\cap_img_0.jpg None


TypeError: image must be numpy array type

In [73]:
img = cv2.imread(1, "C:/Users/raraj/Downloads/annotations/data/image/cap_img_0.jpg")
print(img)

TypeError: Can't convert object to 'str' for 'filename'

# Loggers
**Check this out**

In [ ]:
import json
import sys
from pathlib import Path

import torch
import yaml
from tqdm import tqdm

sys.path.append(str(Path(__file__).parent.parent.parent))  # add utils/ to path
from utils.datasets import LoadImagesAndLabels
from utils.datasets import img2label_paths
from utils.general import colorstr, xywh2xyxy, check_dataset

try:
    import wandb
    from wandb import init, finish
except ImportError:
    wandb = None

WANDB_ARTIFACT_PREFIX = 'wandb-artifact://'


def remove_prefix(from_string, prefix=WANDB_ARTIFACT_PREFIX):
    return from_string[len(prefix):]


def check_wandb_config_file(data_config_file):
    wandb_config = '_wandb.'.join(data_config_file.rsplit('.', 1))  # updated data.yaml path
    if Path(wandb_config).is_file():
        return wandb_config
    return data_config_file


def get_run_info(run_path):
    run_path = Path(remove_prefix(run_path, WANDB_ARTIFACT_PREFIX))
    run_id = run_path.stem
    project = run_path.parent.stem
    model_artifact_name = 'run_' + run_id + '_model'
    return run_id, project, model_artifact_name


def check_wandb_resume(opt):
    process_wandb_config_ddp_mode(opt) if opt.global_rank not in [-1, 0] else None
    if isinstance(opt.resume, str):
        if opt.resume.startswith(WANDB_ARTIFACT_PREFIX):
            if opt.global_rank not in [-1, 0]:  # For resuming DDP runs
                run_id, project, model_artifact_name = get_run_info(opt.resume)
                api = wandb.Api()
                artifact = api.artifact(project + '/' + model_artifact_name + ':latest')
                modeldir = artifact.download()
                opt.weights = str(Path(modeldir) / "last.pt")
            return True
    return None


def process_wandb_config_ddp_mode(opt):
    with open(opt.data) as f:
        data_dict = yaml.load(f, Loader=yaml.SafeLoader)  # data dict
    train_dir, val_dir = None, None
    if isinstance(data_dict['train'], str) and data_dict['train'].startswith(WANDB_ARTIFACT_PREFIX):
        api = wandb.Api()
        train_artifact = api.artifact(remove_prefix(data_dict['train']) + ':' + opt.artifact_alias)
        train_dir = train_artifact.download()
        train_path = Path(train_dir) / 'data/images/'
        data_dict['train'] = str(train_path)

    if isinstance(data_dict['val'], str) and data_dict['val'].startswith(WANDB_ARTIFACT_PREFIX):
        api = wandb.Api()
        val_artifact = api.artifact(remove_prefix(data_dict['val']) + ':' + opt.artifact_alias)
        val_dir = val_artifact.download()
        val_path = Path(val_dir) / 'data/images/'
        data_dict['val'] = str(val_path)
    if train_dir or val_dir:
        ddp_data_path = str(Path(val_dir) / 'wandb_local_data.yaml')
        with open(ddp_data_path, 'w') as f:
            yaml.dump(data_dict, f)
        opt.data = ddp_data_path


class WandbLogger():
    def __init__(self, opt, name, run_id, data_dict, job_type='Training'):
        # Pre-training routine --
        self.job_type = job_type
        self.wandb, self.wandb_run, self.data_dict = wandb, None if not wandb else wandb.run, data_dict
        # It's more elegant to stick to 1 wandb.init call, but useful config data is overwritten in the WandbLogger's wandb.init call
        if isinstance(opt.resume, str):  # checks resume from artifact
            if opt.resume.startswith(WANDB_ARTIFACT_PREFIX):
                run_id, project, model_artifact_name = get_run_info(opt.resume)
                model_artifact_name = WANDB_ARTIFACT_PREFIX + model_artifact_name
                assert wandb, 'install wandb to resume wandb runs'
                # Resume wandb-artifact:// runs here| workaround for not overwriting wandb.config
                self.wandb_run = wandb.init(id=run_id, project=project, resume='allow')
                opt.resume = model_artifact_name
        elif self.wandb:
            self.wandb_run = wandb.init(config=opt,
                                        resume="allow",
                                        project='YOLOR' if opt.project == 'runs/train' else Path(opt.project).stem,
                                        name=name,
                                        job_type=job_type,
                                        id=run_id) if not wandb.run else wandb.run
        if self.wandb_run:
            if self.job_type == 'Training':
                if not opt.resume:
                    wandb_data_dict = self.check_and_upload_dataset(opt) if opt.upload_dataset else data_dict
                    # Info useful for resuming from artifacts
                    self.wandb_run.config.opt = vars(opt)
                    self.wandb_run.config.data_dict = wandb_data_dict
                self.data_dict = self.setup_training(opt, data_dict)
            if self.job_type == 'Dataset Creation':
                self.data_dict = self.check_and_upload_dataset(opt)
        else:
            prefix = colorstr('wandb: ')
            print(f"{prefix}Install Weights & Biases for YOLOR logging with 'pip install wandb' (recommended)")

    def check_and_upload_dataset(self, opt):
        assert wandb, 'Install wandb to upload dataset'
        check_dataset(self.data_dict)
        config_path = self.log_dataset_artifact(opt.data,
                                                opt.single_cls,
                                                'YOLOR' if opt.project == 'runs/train' else Path(opt.project).stem)
        print("Created dataset config file ", config_path)
        with open(config_path) as f:
            wandb_data_dict = yaml.load(f, Loader=yaml.SafeLoader)
        return wandb_data_dict

    def setup_training(self, opt, data_dict):
        self.log_dict, self.current_epoch, self.log_imgs = {}, 0, 16  # Logging Constants
        self.bbox_interval = opt.bbox_interval
        if isinstance(opt.resume, str):
            modeldir, _ = self.download_model_artifact(opt)
            if modeldir:
                self.weights = Path(modeldir) / "last.pt"
                config = self.wandb_run.config
                opt.weights, opt.save_period, opt.batch_size, opt.bbox_interval, opt.epochs, opt.hyp = str(
                    self.weights), config.save_period, config.total_batch_size, config.bbox_interval, config.epochs, \
                                                                                                       config.opt['hyp']
            data_dict = dict(self.wandb_run.config.data_dict)  # eliminates the need for config file to resume
        if 'val_artifact' not in self.__dict__:  # If --upload_dataset is set, use the existing artifact, don't download
            self.train_artifact_path, self.train_artifact = self.download_dataset_artifact(data_dict.get('train'),
                                                                                           opt.artifact_alias)
            self.val_artifact_path, self.val_artifact = self.download_dataset_artifact(data_dict.get('val'),
                                                                                       opt.artifact_alias)
            self.result_artifact, self.result_table, self.val_table, self.weights = None, None, None, None
            if self.train_artifact_path is not None:
                train_path = Path(self.train_artifact_path) / 'data/images/'
                data_dict['train'] = str(train_path)
            if self.val_artifact_path is not None:
                val_path = Path(self.val_artifact_path) / 'data/images/'
                data_dict['val'] = str(val_path)
                self.val_table = self.val_artifact.get("val")
                self.map_val_table_path()
        if self.val_artifact is not None:
            self.result_artifact = wandb.Artifact("run_" + wandb.run.id + "_progress", "evaluation")
            self.result_table = wandb.Table(["epoch", "id", "prediction", "avg_confidence"])
        if opt.bbox_interval == -1:
            self.bbox_interval = opt.bbox_interval = (opt.epochs // 10) if opt.epochs > 10 else 1
        return data_dict

    def download_dataset_artifact(self, path, alias):
        if isinstance(path, str) and path.startswith(WANDB_ARTIFACT_PREFIX):
            dataset_artifact = wandb.use_artifact(remove_prefix(path, WANDB_ARTIFACT_PREFIX) + ":" + alias)
            assert dataset_artifact is not None, "'Error: W&B dataset artifact doesn\'t exist'"
            datadir = dataset_artifact.download()
            return datadir, dataset_artifact
        return None, None

    def download_model_artifact(self, opt):
        if opt.resume.startswith(WANDB_ARTIFACT_PREFIX):
            model_artifact = wandb.use_artifact(remove_prefix(opt.resume, WANDB_ARTIFACT_PREFIX) + ":latest")
            assert model_artifact is not None, 'Error: W&B model artifact doesn\'t exist'
            modeldir = model_artifact.download()
            epochs_trained = model_artifact.metadata.get('epochs_trained')
            total_epochs = model_artifact.metadata.get('total_epochs')
            assert epochs_trained < total_epochs, 'training to %g epochs is finished, nothing to resume.' % (
                total_epochs)
            return modeldir, model_artifact
        return None, None

    def log_model(self, path, opt, epoch, fitness_score, best_model=False):
        model_artifact = wandb.Artifact('run_' + wandb.run.id + '_model', type='model', metadata={
            'original_url': str(path),
            'epochs_trained': epoch + 1,
            'save period': opt.save_period,
            'project': opt.project,
            'total_epochs': opt.epochs,
            'fitness_score': fitness_score
        })
        model_artifact.add_file(str(path / 'last.pt'), name='last.pt')
        wandb.log_artifact(model_artifact,
                           aliases=['latest', 'epoch ' + str(self.current_epoch), 'best' if best_model else ''])
        print("Saving model artifact on epoch ", epoch + 1)

    def log_dataset_artifact(self, data_file, single_cls, project, overwrite_config=False):
        with open(data_file) as f:
            data = yaml.load(f, Loader=yaml.SafeLoader)  # data dict
        nc, names = (1, ['item']) if single_cls else (int(data['nc']), data['names'])
        names = {k: v for k, v in enumerate(names)}  # to index dictionary
        self.train_artifact = self.create_dataset_table(LoadImagesAndLabels(
            data['train']), names, name='train') if data.get('train') else None
        self.val_artifact = self.create_dataset_table(LoadImagesAndLabels(
            data['val']), names, name='val') if data.get('val') else None
        if data.get('train'):
            data['train'] = WANDB_ARTIFACT_PREFIX + str(Path(project) / 'train')
        if data.get('val'):
            data['val'] = WANDB_ARTIFACT_PREFIX + str(Path(project) / 'val')
        path = data_file if overwrite_config else '_wandb.'.join(data_file.rsplit('.', 1))  # updated data.yaml path
        data.pop('download', None)
        with open(path, 'w') as f:
            yaml.dump(data, f)

        if self.job_type == 'Training':  # builds correct artifact pipeline graph
            self.wandb_run.use_artifact(self.val_artifact)
            self.wandb_run.use_artifact(self.train_artifact)
            self.val_artifact.wait()
            self.val_table = self.val_artifact.get('val')
            self.map_val_table_path()
        else:
            self.wandb_run.log_artifact(self.train_artifact)
            self.wandb_run.log_artifact(self.val_artifact)
        return path

    def map_val_table_path(self):
        self.val_table_map = {}
        print("Mapping dataset")
        for i, data in enumerate(tqdm(self.val_table.data)):
            self.val_table_map[data[3]] = data[0]

    def create_dataset_table(self, dataset, class_to_id, name='dataset'):
        # TODO: Explore multiprocessing to slpit this loop parallely| This is essential for speeding up the the logging
        artifact = wandb.Artifact(name=name, type="dataset")
        img_files = tqdm([dataset.path]) if isinstance(dataset.path, str) and Path(dataset.path).is_dir() else None
        img_files = tqdm(dataset.img_files) if not img_files else img_files
        for img_file in img_files:
            if Path(img_file).is_dir():
                artifact.add_dir(img_file, name='data/images')
                labels_path = 'labels'.join(dataset.path.rsplit('images', 1))
                artifact.add_dir(labels_path, name='data/labels')
            else:
                artifact.add_file(img_file, name='data/images/' + Path(img_file).name)
                label_file = Path(img2label_paths([img_file])[0])
                artifact.add_file(str(label_file),
                                  name='data/labels/' + label_file.name) if label_file.exists() else None
        table = wandb.Table(columns=["id", "train_image", "Classes", "name"])
        class_set = wandb.Classes([{'id': id, 'name': name} for id, name in class_to_id.items()])
        for si, (img, labels, paths, shapes) in enumerate(tqdm(dataset)):
            height, width = shapes[0]
            labels[:, 2:] = (xywh2xyxy(labels[:, 2:].view(-1, 4))) * torch.Tensor([width, height, width, height])
            box_data, img_classes = [], {}
            for cls, *xyxy in labels[:, 1:].tolist():
                cls = int(cls)
                box_data.append({"position": {"minX": xyxy[0], "minY": xyxy[1], "maxX": xyxy[2], "maxY": xyxy[3]},
                                 "class_id": cls,
                                 "box_caption": "%s" % (class_to_id[cls]),
                                 "scores": {"acc": 1},
                                 "domain": "pixel"})
                img_classes[cls] = class_to_id[cls]
            boxes = {"ground_truth": {"box_data": box_data, "class_labels": class_to_id}}  # inference-space
            table.add_data(si, wandb.Image(paths, classes=class_set, boxes=boxes), json.dumps(img_classes),
                           Path(paths).name)
        artifact.add(table, name)
        return artifact

    def log_training_progress(self, predn, path, names):
        if self.val_table and self.result_table:
            class_set = wandb.Classes([{'id': id, 'name': name} for id, name in names.items()])
            box_data = []
            total_conf = 0
            for *xyxy, conf, cls in predn.tolist():
                if conf >= 0.25:
                    box_data.append(
                        {"position": {"minX": xyxy[0], "minY": xyxy[1], "maxX": xyxy[2], "maxY": xyxy[3]},
                         "class_id": int(cls),
                         "box_caption": "%s %.3f" % (names[cls], conf),
                         "scores": {"class_score": conf},
                         "domain": "pixel"})
                    total_conf = total_conf + conf
            boxes = {"predictions": {"box_data": box_data, "class_labels": names}}  # inference-space
            id = self.val_table_map[Path(path).name]
            self.result_table.add_data(self.current_epoch,
                                       id,
                                       wandb.Image(self.val_table.data[id][1], boxes=boxes, classes=class_set),
                                       total_conf / max(1, len(box_data))
                                       )

    def log(self, log_dict):
        if self.wandb_run:
            for key, value in log_dict.items():
                self.log_dict[key] = value

    def end_epoch(self, best_result=False):
        if self.wandb_run:
            wandb.log(self.log_dict)
            self.log_dict = {}
            if self.result_artifact:
                train_results = wandb.JoinedTable(self.val_table, self.result_table, "id")
                self.result_artifact.add(train_results, 'result')
                wandb.log_artifact(self.result_artifact, aliases=['latest', 'epoch ' + str(self.current_epoch),
                                                                  ('best' if best_result else '')])
                self.result_table = wandb.Table(["epoch", "id", "prediction", "avg_confidence"])
                self.result_artifact = wandb.Artifact("run_" + wandb.run.id + "_progress", "evaluation")

    def finish_run(self):
        if self.wandb_run:
            if self.log_dict:
                wandb.log(self.log_dict)
            wandb.run.finish()

In [ ]:
from utils.wandb_logging.wandb_utils import WandbLogger, check_wandb_resume

In [ ]:
 # Logging- Doing this before checking the dataset. Might update data_dict
    loggers = {'wandb': None}  # loggers dict
    if rank in [-1, 0]:
        opt.hyp = hyp  # add hyperparameters
        run_id = torch.load(weights, map_location=device).get('wandb_id') if weights.endswith('.pt') and os.path.isfile(weights) else None
        wandb_logger = WandbLogger(opt, Path(opt.save_dir).stem, run_id, data_dict)
        loggers['wandb'] = wandb_logger.wandb
        data_dict = wandb_logger.data_dict
        if wandb_logger.wandb:
            weights, epochs, hyp = opt.weights, opt.epochs, opt.hyp  # WandbLogger might update weights, epochs if resuming

In [ ]:
!labelme

In [12]:
import os

def sep_annot(data_path, new_path):
    if os.path.exists(new_path):
        for data in os.listdir(data_path):
            if data.endswith(".txt") and os.path.exists(os.path.join(data_path,f'{data.split(".")[0]}.jpg')):
                os.replace(os.path.join(data_path, data), os.path.join(data_path))
                os.replace(os.path.join(data_path, f'{data.split(".")[0]}.jpg'), os.path.join(data_path))
    else:
        os.mkdir(new_path)
        sep_annot(data_path, new_path)

In [14]:
sep_annot("C:/Users/raraj/Downloads/annotations/shoplifting_frames", "C:/Users/raraj/Downloads/annotations/comleted_shoplifting_frames")

PermissionError: [WinError 5] Access is denied: 'C:/Users/raraj/Downloads/annotations/shoplifting_frames\\2988.txt' -> 'C:/Users/raraj/Downloads/annotations/shoplifting_frames'

In [27]:
a = {'a':'1', 'b':'2', 'c':'3', 'd':'4'}
# list(map(lambda a.items() : a, a.items()))

In [28]:
list(map(lambda a: a+2, list(range(10))))

[2, 3, 4, 5, 6, 7, 8, 9, 10, 11]

In [38]:
values = a.values()
values

dict_values(['1', '2', '3', '4'])

In [40]:
values = list(map(lambda x: a[x]+'2', values))
a

SyntaxError: cannot assign to function call (1744704390.py, line 1)

In [47]:
a = {"shape_attributes" : {"name": "rect", "x": 184, "y": 298, "width": 14, "height": 15}}

In [48]:
a

{'shape_attributes': {'name': 'rect',
  'x': 184,
  'y': 298,
  'width': 14,
  'height': 15}}

In [50]:
b = a.values()

In [51]:
b

dict_values([{'name': 'rect', 'x': 184, 'y': 298, 'width': 14, 'height': 15}])

In [10]:
pip install mysqlclient

  Using cached mysqlclient-2.2.0-cp39-cp39-win_amd64.whl (200 kB)
Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'C:\Users\raraj\computer_vision\ComVis\scripts\python.exe -m pip install --upgrade pip' command.


In [11]:
import MySQLdb

# Data Preparation for action recognition

In [2]:
import mediapipe as mp

draw = mp.solutions.drawing_utils
holistic = mp.solutions.holistic

In [3]:
import cv2
import os
import matplotlib.pyplot as plt


def make_dirs(actions, samples_per_action, root_path, data_path):
    for action in actions:
        for sample in range(samples_per_action):
            try:
                os.makedirs(os.path.join(root_path, data_path, action, f"{sample}"))
            except FileExistsError:
                pass

def read_frames(data_path, frames_per_sample, source = 0):
    if os.path.exists(data_path):
        with holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as kp:
            cap = cv2.VideoCapture(source)
            for action in os.listdir(data_path):
                for sample in os.listdir(os.path.join(data_path, action)):
                    for frame_no in range(frames_per_sample):
                        success, frame = cap.read()
                        if success:
                            result_kp = kp.process(frame)
                            frame.flags.writeable = True
                            draw.draw_landmarks(frame, result_kp.pose_landmarks, holistic.POSE_CONNECTIONS)
                            if frame_no== 0:
                                cv2.putText(frame, "Started Collecting frames", (120,200), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 4, cv2.LINE_AA)
                                cv2.putText(frame, f"Action: {action}\nSample_no: {sample}\nFrame_no: {frame_no}", (120,200), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,0,0), 4, cv2.LINE_AA)
                            else:
                                cv2.putText(frame, f"Action: {action}\nSample_no: {sample}\nFrame_no: {frame_no}", (120,200), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,0,0), 4, cv2.LINE_AA)
                            cv2.imshow("Source feed", frame)
            cap.release()
            cv2.destroyAllWindows()
    else:
        make_dirs(["running","sneaking","hiding_item","picking_item"], 50, "D:/annotations", "D:/annotations/Shoplifting_action_recognition")
        print("required directories created\n starting to record the data")
        read_frames(data_path, frames_per_sample)

TypeError: 'numpy._DTypeMeta' object is not subscriptable

In [ ]:
read_frames("D:/annotations/Shoplifting_action_recognition", 50, source = 0)

In [11]:
pip list

Package                       Version
----------------------------- --------------------
absl-py                       1.2.0
aiohttp                       3.8.1
aiosignal                     1.2.0
alabaster                     0.7.12
albumentations                1.3.1
anaconda-client               1.9.0
anaconda-navigator            2.3.0
anaconda-project              0.10.2Note: you may need to restart the kernel to use updated packages.
anyio                         3.5.0
appdirs                       1.4.4
argon2-cffi                   21.3.0
argon2-cffi-bindings          21.2.0
arrow                         1.2.2
astroid                       2.6.6
astropy                       5.0.4
asttokens                     2.0.5
astunparse                    1.6.3
async-timeout                 4.0.1
atomicwrites                  1.4.0
attrs                         21.4.0
Automat                       20.2.0
autopep8                      1.6.0
Babel                         2.9.1



backcall                      0.2.0
backports.functools-lru-cache 1.6.4
backports.tempfile            1.0
backports.weakref             1.0.post1
bcrypt                        3.2.0
beautifulsoup4                4.11.1
binaryornot                   0.4.4
bitarray                      2.4.1
bkcharts                      0.2
black                         19.10b0
bleach                        4.1.0
bokeh                         2.4.2
boto3                         1.21.32
botocore                      1.24.32
Bottleneck                    1.3.4
brotlipy                      0.7.0
cachetools                    4.2.2
certifi                       2021.10.8
cffi                          1.15.0
chardet                       4.0.0
charset-normalizer            2.0.4
click                         8.0.4
cloudpickle                   2.0.0
clyent                        1.2.2
colorama                      0.4.4
colorcet                      2.0.6
comtypes                      1.1.10
conda         

In [2]:
pip install --upgrade opencv-python

  Attempting uninstall: opencv-python
Note: you may need to restart the kernel to use updated packages.


    Found existing installation: opencv-python 4.6.0.66
    Uninstalling opencv-python-4.6.0.66:
      Successfully uninstalled opencv-python-4.6.0.66


In [2]:
!labelme